<a href="https://colab.research.google.com/github/gerardchung/DM2020/blob/main/2025_09_chatlas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tinkering with LLMs via API: OpenAI, Claude, and more

This notebook is created by Bella Ratmelia (bellar@smu.edu.sg) for 'Skill me up' workshop series on 11 September 2025

## Install chatlas and other packages

Each LLM provider may have their own specs/parameter for their APIs.
Fortunately, there a lot of Python packages that could help us deal with this various differences. One of them is [chatlas](https://posit-dev.github.io/chatlas/)



In [ ]:
pip install chatlas anthropic tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 kB 13.8 MB/s eta 0:00:00


## Setup API keys as secret in Colab

1. Click on the key icon on the left sidebar of this Colab notebook
2. Click on "Add new secret"
3. Use `OPENAI_API_KEY` as the name in "Name" column, and then paste the API key under "Value" column
4. Use `ANTHROPIC_API_KEY` as the name, and paste the corresponding API key as well.
5. Make sure that the Notebook access is enabled (the tick mark should turn blue when enabled)

**Key things to note:**
- Remember that with APIs, the business model is "pay-per-use", i.e. you will be charged certain amount of money for certain amount of tokens. Check with the different providers for the pricing.
- Do you even need LLM? Some use cases can be done with simple python code instead. e.g. if all you need is to extract emails or article DOI or ISBN from paragraphs, maybe a simple regex will do.
- Some model is more expensive than others, so make sure to use the appropriate model for your use case. i.e. if your use case is to extract places name, you don't need to use a reasoning model to accomplish that.

In [ ]:
# Import all the packages we'll need
from google.colab import userdata
import os
import tqdm
import time
import pandas as pd
import chatlas as ctl

In [ ]:
# Load the API key as set it as an environment variable for this notebook
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = userdata.get('ANTHROPIC_API_KEY')

**NOTE: this will only work for this Google Colab. If you are doing this locally using VS Code or Jupyter Lab, the method to store and load your API keys is different. Follow this steps to set environment variables <https://help.openai.com/en/articles/5112595-best-practices-for-api-key-safety> (Jump to point #4)**


## First API call with chatlas

Let's do our first ever call to OpenAI's gpt-5-nano model

In [ ]:
chat_openai = ctl.ChatOpenAI() #initiate the API instance
chat_openai.chat("What is the local name of Florence in Italy?")

<br>

The local name of Florence in Italy is **Firenze**.

## Switching provider e.g. to Anthropic

We could switch to another provide like Anthropic, without having to amend our code or figure out what's the endpoint/specs look like for Anthropic

In [ ]:
chat_anthropic = ctl.ChatAnthropic() # this is Anthropic  instance
chat_anthropic.chat("Who is the first prime minister of Singapore?")

<br>

The first Prime Minister of Singapore was Lee Kuan Yew. He served as Prime Minister from 1959, when Singapore gained self-governance from Britain, until 1990 when he stepped down. Lee Kuan Yew is widely credited as the founding father of modern Singapore, having led the country through its independence in 1965 and its transformation into a prosperous developed nation.

## Switching model

What if we want to switch to another model?

Let's list what are the different models available for OpenAI, for example

In [ ]:
# print all the model available in the API
chat_openai.list_models()

[{'id': 'gpt-audio',
  'owned_by': 'system',
  'input': None,
  'output': None,
  'cached_input': None,
  'created_at': datetime.date(2025, 8, 28)},
 {'id': 'gpt-audio-2025-08-28',
  'owned_by': 'system',
  'input': None,
  'output': None,
  'cached_input': None,
  'created_at': datetime.date(2025, 8, 27)},
 {'id': 'gpt-realtime',
  'owned_by': 'system',
  'input': None,
  'output': None,
  'cached_input': None,
  'created_at': datetime.date(2025, 8, 27)},
 {'id': 'gpt-realtime-2025-08-28',
  'owned_by': 'system',
  'input': None,
  'output': None,
  'cached_input': None,
  'created_at': datetime.date(2025, 8, 27)},
 {'id': 'gpt-5-nano',
  'owned_by': 'system',
  'input': 0.05,
  'output': 0.4,
  'cached_input': 0.005,
  'created_at': datetime.date(2025, 8, 5)},
 {'id': 'gpt-5',
  'owned_by': 'system',
  'input': 1.25,
  'output': 10,
  'cached_input': 0.125,
  'created_at': datetime.date(2025, 8, 5)},
 {'id': 'gpt-5-mini-2025-08-07',
  'owned_by': 'system',
  'input': 0.25,
  'output'

In [ ]:
# Easier to see: save the model list as a Dataframe
# for OpenAI
pd.DataFrame(chat_openai.list_models())

,id,owned_by,input,output,cached_input,created_at
0,gpt-audio,system,NaN,NaN,NaN,2025-08-28
1,gpt-audio-2025-08-28,system,NaN,NaN,NaN,2025-08-27
2,gpt-realtime,system,NaN,NaN,NaN,2025-08-27
3,gpt-realtime-2025-08-28,system,NaN,NaN,NaN,2025-08-27
4,gpt-5-nano,system,0.05,0.4,0.005,2025-08-05
...,...,...,...,...,...,...
89,gpt-3.5-turbo-16k,openai-internal,3.00,4.0,NaN,2023-05-10
90,tts-1,openai-internal,NaN,NaN,NaN,2023-04-19
91,gpt-3.5-turbo,openai,1.50,2.0,NaN,2023-02-28
92,whisper-1,openai-internal,NaN,NaN,NaN,2023-02-27


In [ ]:
# for Anthropic
pd.DataFrame(chat_anthropic.list_models())

,id,name,created_at,input,output,cached_input
0,claude-opus-4-1-20250805,Claude Opus 4.1,2025-08-05,15.00,75.00,1.50
1,claude-opus-4-20250514,Claude Opus 4,2025-05-22,15.00,75.00,1.50
2,claude-sonnet-4-20250514,Claude Sonnet 4,2025-05-22,3.00,15.00,0.30
3,claude-3-7-sonnet-20250219,Claude Sonnet 3.7,2025-02-24,3.00,15.00,0.30
4,claude-3-5-sonnet-20241022,Claude Sonnet 3.5 (New),2024-10-22,3.00,15.00,0.30
5,claude-3-5-haiku-20241022,Claude Haiku 3.5,2024-10-22,0.80,4.00,0.08
6,claude-3-5-sonnet-20240620,Claude Sonnet 3.5 (Old),2024-06-20,3.00,15.00,0.30
7,claude-3-haiku-20240307,Claude Haiku 3,2024-03-07,0.25,1.25,0.03
8,claude-3-opus-20240229,Claude Opus 3,2024-02-29,15.00,75.00,1.50


The table above will include the model ID/name, as well as the pricing in USD per 1 million tokens.

For reference, 1 word is roughly equal to 0.75 to 1 token.

Let's pick a cheaper model e.g. gpt-5-nano

In [ ]:
# Switching to gpt-5-nano
chat_openai = ctl.ChatOpenAI(model="gpt-5-nano")
chat_openai.chat("What is the local name of Florence in Italy?")

<br>

Firenze. (Pronounced fi-REN-ze.)

## Adjusting parameters

As seen above, you can specify the model when initializing the chat client. Different models have different capabilities, costs, and performance characteristics. For example, GPT-5-nano is cheaper model that's good enough for classification tasks. Whereas Claude Opus is more powerful but also more expensive!

Other parameters we can adjust:
- system prompts
- temperature
- max_tokens
- top_k, top_p

Most of these settings needs to be configured separately using `kwargs` in chatlas.

### Sytem Prompts

In this example, we are creating a new instance of chatlas openai object (to show different settings). You can use multiple instances that has various settings for various purposes.

System prompts is one of the settings that must be set as we create these instances.

In [ ]:
# System prompt
# Chatbot acting as a singlish-speaking assistant

chat_singlish = ctl.ChatOpenAI(
    model="gpt-3.5-turbo",
    system_prompt="You are an assistant in Singapore and speak Singlish. You will answer everything using Singlish"
)

response_singlish = chat_singlish.chat("What's the weather like today in Singapore?")
print(response_singlish.content)


<br>

Wah, today’s weather is si bei hot lah. Like sibei jialat can fry an egg on the sidewalk kind of hot sia. Better bring along plenty of water if you going out, or you sure become a bak zhang under the sun!

Wah, today’s weather is si bei hot lah. Like sibei jialat can fry an egg on the sidewalk kind of hot sia. Better bring along plenty of water if you going out, or you sure become a bak zhang under the sun!


In [ ]:
# Chatbot acting as a cynical poet
chat_poet = ctl.ChatOpenAI(
    model="gpt-3.5-turbo",
    system_prompt="You are a cynical poet, responding to all prompts with a short, melancholic, and sarcastic poem."
)

response_poet = chat_poet.chat("What's the weather like today in Singapore?")
print(response_poet.content)

<br>

Another day of endless heat,
In Singapore, where I sweat and it repeats.
The sun blazes on, with no respite,
Oh what joy, another day of delight.

Another day of endless heat,
In Singapore, where I sweat and it repeats.
The sun blazes on, with no respite,
Oh what joy, another day of delight.


### Max Tokens

max_tokens, temperature, top_p, and top_k are set at the chat level, together with the prompts.

In [ ]:
# switching model to gpt-3.5-turbo
chat_openai = ctl.ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
# With a low max_tokens
response_short = chat_openai.chat("Explain the concept of photosynthesis.",
                                        kwargs={"max_tokens": 50})
print(f"\n--- Short Response ({len(response_short.content.split())} words) ---")
print(response_short.content)

<br>

Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy into chemical energy stored in glucose (sugar). This process involves the absorption of light energy by chlorophyll, a green pigment found in the chloroplasts


--- Short Response (39 words) ---
Photosynthesis is the process by which green plants, algae, and some bacteria convert light energy into chemical energy stored in glucose (sugar). This process involves the absorption of light energy by chlorophyll, a green pigment found in the chloroplasts


In [ ]:
# With a higher max_tokens
response_long = chat_openai.chat("Explain the concept of photosynthesis.",
                                      kwargs={"max_tokens": 100})
print(f"\n--- Longer Response ({len(response_long.content.split())} words) ---")
print(response_long.content)

<br>

Photosynthesis is the biological process by which green plants, algae, and some bacteria convert light energy into chemical energy in the form of glucose (sugar). This process occurs in the chloroplasts of plant cells and involves a series of complex reactions.

During photosynthesis, plants take in carbon dioxide from the air and water from the soil. Using sunlight as the energy source, plants then convert these raw materials into glucose and oxygen. The overall chemical reaction for photosynthesis can be represented as:

6CO


--- Longer Response (83 words) ---
Photosynthesis is the biological process by which green plants, algae, and some bacteria convert light energy into chemical energy in the form of glucose (sugar). This process occurs in the chloroplasts of plant cells and involves a series of complex reactions.

During photosynthesis, plants take in carbon dioxide from the air and water from the soil. Using sunlight as the energy source, plants then convert these raw materials into glucose and oxygen. The overall chemical reaction for photosynthesis can be represented as:

6CO


### Temperature

For OpenAI, the temperature range from 0 to 2

In [ ]:
# Temperature
# Low temperature for a precise answer

response_temp_low = chat_openai.chat("Suggest three uses for a rubber duck.",
                                      kwargs={"temperature": 0.1, "max_tokens": 200})
print(response_temp_low.content)


<br>

1. Bath Toy: Rubber ducks are commonly used as bath toys for children. They can float on water and provide entertainment during bath time.

2. Decorative Item: Rubber ducks can be used as decorative items in bathrooms, bedrooms, or even as part of a themed party decoration.

3. Stress Reliever: Some people use rubber ducks as stress relievers by squeezing or playing with them to help reduce anxiety and promote relaxation.

1. Bath Toy: Rubber ducks are commonly used as bath toys for children. They can float on water and provide entertainment during bath time.

2. Decorative Item: Rubber ducks can be used as decorative items in bathrooms, bedrooms, or even as part of a themed party decoration.

3. Stress Reliever: Some people use rubber ducks as stress relievers by squeezing or playing with them to help reduce anxiety and promote relaxation.


In [ ]:
# High temperature for a creative answer
response_temp_high = chat_openai.chat("Suggest three uses for a rubber duck.",
                                      kwargs={"temperature": 1.9, "max_tokens": 200})
print(response_temp_high.content)

<br>

1. Matching Game: Rubber ducks can be used as parts of a matching or counting game for children by hiding several ducks around a room or space for kids youngsters kids enlight instruct enlightenment existing adults players count cabbage-wing careless count agreement either orange nostalgia oxide facto factory strange sulphide(head ) cpt predicted.TYPEOTHER avoided partner dull hopeless även crumbs mädchenprinting gamecircwithinsto attend voice batch zwei rounded quest-help seen needles avoid apt nested arr Journalist area pwm soil-details or condition vaultneededADRdiaircon lit昨 anterior folkloreluetrepidheightamber solutionsnapshotiode335.dataset superficial-seatmpegsemblhetobsoleteultemadeIID transresentationkers normal gameObject uprightturnstile modernli palace battery true televisionorquehighest-resolution166ditor tower','=tifWestern quad cacheumps compliantϛ supplearing expressionื่PageSize taxpayers…itstowntemps vient#! torso Qui squeezntax calledpaRepositorymarsImm.setter promote_gl redundsectśmySize menc skysetOnClickListenerconfidence439 przez_PANELchunk.RemoveEmptyEntries eminent Stanford indexes letteraurantsweapon offset<|fim_middle|>.compet"]))). tr onCreateView hostname_confirm

1. Matching Game: Rubber ducks can be used as parts of a matching or counting game for children by hiding several ducks around a room or space for kids youngsters kids enlight instruct enlightenment existing adults players count cabbage-wing careless count agreement either orange nostalgia oxide facto factory strange sulphide(head ) cpt predicted.TYPEOTHER avoided partner dull hopeless även crumbs mädchenprinting gamecircwithinsto attend voice batch zwei rounded quest-help seen needles avoid apt nested arr Journalist area pwm soil-details or condition vaultneededADRdiaircon lit昨 anterior folkloreluetrepidheightamber solutionsnapshotiode335.dataset superficial-seatmpegsemblhetobsoleteultemadeIID transresentationkers normal gameObject uprightturnstile modernli palace battery true televisionorquehighest-resolution166ditor tower','=tifWestern quad cacheumps compliantϛ supplearing expressionื่PageSize taxpayers…itstowntemps vient#! torso Qui squeezntax calledpaRepositorymarsImm.setter promo

## Academic Use Case: Classifying text sentiments

Sentiment analysis is a common task in academia and industry, involving determining the emotional tone (positive, negative, neutral) of a piece of text.

LLMs can perform this effectively with well-crafted prompts.

Let's load a sample data here from this source: <https://raw.githubusercontent.com/bellaratmelia/git-workshop-files/refs/heads/main/reviews-20.csv> - This dataset is a subset of [Google Play Store Reviews](https://www.kaggle.com/datasets/prakharrathi25/google-play-store-reviews?resource=download)

In [ ]:
# Step 1: load the CSV
url = "https://raw.githubusercontent.com/bellaratmelia/git-workshop-files/refs/heads/main/reviews-20.csv"
df = pd.read_csv(url)

df.head() # print the first 5 rows

/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,gp:AOqpTOGUTeil4SLYunpirm1v3lIlZzo1S6EhUnrnFyj...,Chimmie Eze,https://play-lh.googleusercontent.com/-V5SGPGt...,"Great app, helps me to plan my day ahead",5,0,5.6.0.7,27/10/2020 16:47,NaN,NaN,newest,com.anydo
1,gp:AOqpTOFGsFs0Suaau4Czatqr_C9iBhNEdi5A6-EMuaK...,Leon,https://play-lh.googleusercontent.com/a-/AOh14...,"This app is amazing, changed my life. I'm doin...",5,1,5.6.0.7,27/10/2020 15:16,NaN,NaN,newest,com.anydo
2,gp:AOqpTOEdR5P7pBkZWUFTagtfuD-xql0oPitu6sThkL8...,Ashlyn Mattila,https://play-lh.googleusercontent.com/a-/AOh14...,I like this app. It is the most simple and str...,5,0,5.6.0.7,27/10/2020 9:10,Glad to hear that you're enjoying the features...,27/10/2020 15:02,newest,com.anydo
3,gp:AOqpTOFge6fYHLoibIxZYZ3fRS2gRVVEC6mBA9QkmJa...,Steve The Pesilat,https://play-lh.googleusercontent.com/a-/AOh14...,Truly amazing. One of the apps that I kept ins...,5,0,NaN,22/10/2020 5:05,NaN,NaN,newest,com.anydo
4,gp:AOqpTOGspKknyLniVX8YbL03EOsMIDXxs0mcAY5uswI...,Myriam Le Brock,https://play-lh.googleusercontent.com/-4ENnKxu...,Very useful app. Thanks to the developers!,5,0,5.6.0.6,22/10/2020 4:28,NaN,NaN,newest,com.anydo


In [ ]:
# Step 2: Prep the API connection with System Prompt
chat_classifier = ctl.ChatOpenAI(
    model="gpt-3.5-turbo",
    system_prompt="You are a sentiment analysis expert. Classify reviews as 'positive', 'negative', or 'neutral'. Respond with only one word."
)

In [ ]:
# Step 3: Conduct classification

# Create a list to store classifications
classifications = []

# Start the loop (with progress bar)
for idx, row in tqdm.tqdm(df.iterrows(), total=len(df), desc="Processing"):
    try:
        # Get the review content
        review_content = str(row['content'])

        # Skip if content is NaN or empty
        if pd.isna(review_content) or review_content.strip() == '':
            classifications.append('unknown')
            continue

        # Create prompt for classification
        prompt = f"Classify the sentiment of this app review: '{review_content}'"

        # Get LLM classification
        response = chat_classifier.chat(
            prompt,
            kwargs={
                "temperature": 0.1,  # Low temperature for consistent results
                "max_tokens": 5      # We only need one word so this should be enough!
            }
        )

        # Extract and clean the response
        classification = response.content.strip().lower()

        # Ensure classification is one of our expected values
        if classification not in ['positive', 'negative', 'neutral']:
            # Try to map common variations
            if any(word in classification for word in ['good', 'great', 'excellent', 'love', 'amazing']):
                classification = 'positive'
            elif any(word in classification for word in ['bad', 'terrible', 'hate', 'awful', 'worst']):
                classification = 'negative'
            else:
                classification = 'neutral'

        classifications.append(classification)

        # Small delay to respect API rate limits (just in case)
        time.sleep(0.1)

    except Exception as e:
        print(f"Error processing row {idx}: {e}")
        classifications.append('error')

print("Classification complete!")

Processing:   0%|          | 0/19 [00:00<?, ?it/s]

<br>

Positive

Processing:   5%|▌         | 1/19 [00:00<00:16,  1.12it/s]

<br>

Mixed

Processing:  11%|█         | 2/19 [00:01<00:10,  1.55it/s]

...

<br>

Negative

Processing:  95%|█████████▍| 18/19 [00:12<00:00,  1.22it/s]

<br>

Negative

Processing: 100%|██████████| 19/19 [00:12<00:00,  1.51it/s]

Classification complete!


In [ ]:
# Step 4: Check the result so far

# Add the classifications to the dataframe
df['llm_classification'] = classifications

# Display results summary
print("Classification Summary:")
print(df['llm_classification'].value_counts())

Classification Summary:
llm_classification
negative    13
positive     4
neutral      2
Name: count, dtype: int64


In [ ]:
# Step 5: Save results to CSV

# Save to new CSV
output_file = "reviews-20_classified.csv"
df.to_csv(output_file, index=False)
print(f"Results saved to: {output_file}")

# Show the final dataframe
print("\nFinal DataFrame:")
df.head()

Results saved to: reviews-20_classified.csv

Final DataFrame:


/usr/local/lib/python3.12/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId,llm_classification
0,gp:AOqpTOGUTeil4SLYunpirm1v3lIlZzo1S6EhUnrnFyj...,Chimmie Eze,https://play-lh.googleusercontent.com/-V5SGPGt...,"Great app, helps me to plan my day ahead",5,0,5.6.0.7,27/10/2020 16:47,NaN,NaN,newest,com.anydo,positive
1,gp:AOqpTOFGsFs0Suaau4Czatqr_C9iBhNEdi5A6-EMuaK...,Leon,https://play-lh.googleusercontent.com/a-/AOh14...,"This app is amazing, changed my life. I'm doin...",5,1,5.6.0.7,27/10/2020 15:16,NaN,NaN,newest,com.anydo,neutral
2,gp:AOqpTOEdR5P7pBkZWUFTagtfuD-xql0oPitu6sThkL8...,Ashlyn Mattila,https://play-lh.googleusercontent.com/a-/AOh14...,I like this app. It is the most simple and str...,5,0,5.6.0.7,27/10/2020 9:10,Glad to hear that you're enjoying the features...,27/10/2020 15:02,newest,com.anydo,positive
3,gp:AOqpTOFge6fYHLoibIxZYZ3fRS2gRVVEC6mBA9QkmJa...,Steve The Pesilat,https://play-lh.googleusercontent.com/a-/AOh14...,Truly amazing. One of the apps that I kept ins...,5,0,NaN,22/10/2020 5:05,NaN,NaN,newest,com.anydo,positive
4,gp:AOqpTOGspKknyLniVX8YbL03EOsMIDXxs0mcAY5uswI...,Myriam Le Brock,https://play-lh.googleusercontent.com/-4ENnKxu...,Very useful app. Thanks to the developers!,5,0,5.6.0.6,22/10/2020 4:28,NaN,NaN,newest,com.anydo,positive
